# Tension Terminator Yolo Transferlearning

## Set global variables

In [1]:
# Set Variables
BASE_MODEL = "yolov8s"
TRAIN_EPOCHS = 100
TRAIN_SIZE = 320

TT_PROJECTNAME = "TensionTerminator"

## Mount Google Drive to backup files

In [2]:
# Mount Google Drive
from google.colab import drive
import os

drive.mount('/content/drive')

google_drive_target_folder = '/content/drive/My Drive/'+TT_PROJECTNAME+'/'
# Check if the target folder exists in Google Drive, create if it doesn't
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)
    print(f"Created folder: {google_drive_target_folder}")
print("Google Drive Target Folder: " + google_drive_target_folder)

Mounted at /content/drive
Google Drive Target Folder: /content/drive/My Drive/TensionTerminator/


# Install Dependencies

In [3]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.6/646.6 kB 11.8 MB/s eta 0:00:00


In [4]:
!pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstalled opencv-python-headless-4.8.1.78
  Attempting uninstall: idna
    Found existin

## Python imports

In [5]:
import os
import random
import yaml
import shutil
from ultralytics import YOLO
from google.colab import userdata, files
from datetime import datetime
from ultralytics.utils.benchmarks import benchmark
from roboflow import Roboflow

## Loading dataset to train on

In [6]:
secret_key = userdata.get('rb-key')

rf = Roboflow(api_key=secret_key)
project = rf.workspace("juzaworkspace-wzpkn").project("tensionterminatordetectionv2")
dataset = project.version(5).download("yolov8")

DATASET_DIR = dataset.location
DATA_YAML = DATASET_DIR + "/data.yaml"
print("\n\info")
print("Data saved to: " + DATASET_DIR)
print("Data YAML: " + DATA_YAML)

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.221, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to TensionTerminatorDetectionV2-5 in yolov8:: 100%|██████████| 2437/2437 [00:01<00:00, 1857.96it/s]



\info
Data saved to: /content/TensionTerminatorDetectionV2-5
Data YAML: /content/TensionTerminatorDetectionV2-5/data.yaml


There is a bug in the data.yaml of robfolow yolov8. This script corrects it.

In [7]:
# corrects the folders in the data.yaml

# Path to the YAML file
yaml_file_path = DATASET_DIR + 'data.yaml'

# Read the YAML file
with open(DATA_YAML) as file:
    data = yaml.safe_load(file)

# Update the paths
data['train'] = DATASET_DIR + '/train/images'
data['val'] = DATASET_DIR + '/valid/images'
data['test'] = DATASET_DIR + '/test/images'

# Write the updated data back to the YAML file
with open(DATA_YAML, 'w') as file:
    yaml.dump(data, file)

print("Updated data.yaml successfully.")


Updated data.yaml successfully.


## Train a new model

In [ ]:
# Variables already set, but u can run to re-train just this cell and modify the vars (not recommended!)
#TRAIN_MODEL = "yolov8s"
#TRAIN_EPOCHS = 1
#TRAIN_SIZE = 320

# Load a yolo model as base
modelpt = BASE_MODEL + ".pt"
model = YOLO(modelpt)

# train the model
model.train(data=DATA_YAML, epochs=TRAIN_EPOCHS, imgsz=TRAIN_SIZE)

100%|██████████| 21.5M/21.5M [00:00<00:00, 333MB/s]


Ultralytics YOLOv8.0.221 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/TensionTerminatorDetectionV2-5/data.yaml, epochs=100, patience=50, batch=16, imgsz=320, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_w

100%|██████████| 755k/755k [00:00<00:00, 114MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 6.23M/6.23M [00:00<00:00, 296MB/s]


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     41/100      1.28G     0.5911     0.3445     0.8947         14        320: 100%|██████████| 73/73 [00:33<00:00,  2.18it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.02it/s]

                   all         42         99      0.946      0.976      0.955      0.715



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     42/100      1.38G     0.5962     0.3403     0.8989         73        320:  81%|████████  | 59/73 [00:33<00:12,  1.16it/s]

## Save the model to Google Drive

In [ ]:
# Get latest run dir
base_dir = '/content/runs/detect'
latest_run_dir = max([os.path.join(base_dir, d) for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))], key=os.path.getmtime)
print(latest_run_dir)

# modify if there is an error
#latest_run_dir = "/content/runs/detect/train3"

In [ ]:
# Define the base directory where the runs are stored
TRAINED_WEIGHTS = latest_run_dir + "/weights/" + "best.pt"

# Construct the path to the best weights file
print("Original trained weights path: ", TRAINED_WEIGHTS)

# Get the current timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Construct the new file name with timestamp
new_file_name = f"TTmodel2_{BASE_MODEL}_img{TRAIN_SIZE}_{timestamp}.pt"
new_file_path = os.path.join(latest_run_dir, 'weights', new_file_name)

# Copy the file with timestamp in the same directory
shutil.copy2(TRAINED_WEIGHTS, new_file_path)
print("New file path with timestamp: ", new_file_path)

# Make sure the target folder exists
if not os.path.exists(google_drive_target_folder):
    os.makedirs(google_drive_target_folder)

# Construct the target file path in Google Drive
google_drive_target_path = os.path.join(google_drive_target_folder, new_file_name)

# Copy the file to Google Drive
shutil.copy2(new_file_path, google_drive_target_path)
print("Model file copied to Google Drive: ", google_drive_target_path)

## Evaluate Performance

In [ ]:
# Evaluate the model's performance on the validation set
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Testing on 3 random files

In [ ]:
# Define the path to the test images directory
test_images_dir = DATASET_DIR + "/test/images"

# Get a list of all files in the test images directory
all_test_images = os.listdir(test_images_dir)

# Randomly select 3 images
random_test_images = random.sample(all_test_images, 3)

# Predict on each of the 3 random images
for image_name in random_test_images:
    image_path = os.path.join(test_images_dir, image_name)
    results = model(image_path)  # Perform prediction
    print(f"Results for {image_name}:")
    print(results)  # or use results.show() or results.save() based on your requirement

## Export the file to OAK-D

In [ ]:
print("\nGo to Tool at http://tools.luxonis.com/ and make a export of the downloaded model file " + new_file_name + " for the OAK-Camera.")
print("\nPro Tipps to use the Luxonis tool: \n  *) choose the correct Yolo Version (example: YoloV8 detection only)\n  *) choose the correct image shape (example: "+ str(TRAIN_SIZE) + ")\n  *) edit the advanced options, choose 6 shaves and deactivate OpenVINO 2021.4 checkbox.\n\n")

files.download(new_file_path)

In [ ]:
# for other use cases u can follow the ONNX path https://docs.luxonis.com/en/latest/pages/model_conversion/#converting-model-to-myriadx-blob
# converts the model to onnx
'''
path = model.export(format="onnx")  # export the model to ONNX format
TRAINED_WEIGHTS_ONNX = latest_run_dir + "/weights/" + "best.onnx"
files.download(TRAINED_WEIGHTS_ONNX)
'''

In [ ]:
# Benchmark on GPU
# deactivated, cause needs a lot of time
'''
benchmark(model=TRAINED_WEIGHTS, data=DATA_YAML, imgsz=TRAIN_SIZE, half=False, device=0)
'''

# END